In [1]:
from SimPEG import Mesh
from simpegEM1D import *
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/sgkang/anaconda2/envs/simpeg/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['pi']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Set up for EM1D program

## Step1: Generate mesh

In [2]:
nearthick = np.logspace(-1, 1, 5)
deepthick = np.logspace(1, 2, 10)
hx = np.r_[nearthick, deepthick]
mesh1D = Mesh.TensorMesh([hx], [0.])
depth = -mesh1D.gridN[:-1]
LocSigZ = -mesh1D.gridCC
nlay = depth.size
topo = np.r_[0., 0., 100.]

## Step2: Set survey parameters

In [6]:
FDsurvey = EM1DSurveyFD(
    rx_location = np.array([0., 0., 100.+1e-5]),
    srcLoc = np.array([0., 0., 100.+1e-5]),
    fieldtype = 'secondary',
    rxType = 'Hz',
    srcType = 'VMD',
    offset = np.r_[8.],
    depth = depth,
    topo = topo,
    LocSigZ = LocSigZ,
    frequency = np.logspace(-2, 8, 61)
)


Exception: rxLoc attr is not recognized

## Step3: Set mappings (Exponential and Cole-Cole maps)

### Exponential map

### $$ m = log(\sigma) $$

### $$ M_{exp}(m) = e^{m}$$

### Cole-Cole map

$$ m(\sigma_{\infty}, \eta, \tau, c; \omega) = \sigma_{\infty} - \sigma_{\infty}\eta \frac{1}{1+(1-\eta)(\imath\omega\tau)^c}$$

In [ ]:
sig_half = 1e0
chi_half = 0.
options = {'Frequency': FDsurvey.frequency, 'tau': np.ones(nlay)*0.002, 'eta':np.ones(nlay)*0.5, 'c':np.ones(nlay)*1.}
Expmap = BaseEM1D.BaseEM1DMap(mesh1D)
Colemap = BaseEM1D.BaseColeColeMap(mesh1D, **options)*Expmap

In [ ]:
m_1D = np.log(np.ones(nlay)*sig_half)

In [ ]:
%%time
WT0, WT1, YBASE = DigFilter.LoadWeights()
options = {'WT0': WT0, 'WT1': WT1, 'YBASE': YBASE}
prob = EM1D.EM1D(mesh1D, sigmaMap=Expmap, **options)

if prob.ispaired:
    prob.unpair()
if FDsurvey.ispaired:
    FDsurvey.unpair()
prob.pair(FDsurvey)
prob.CondType = 'Real'
prob.chi = np.zeros(FDsurvey.nlay)
Hz = prob.fields(m_1D)

In [ ]:
%%time
options = {'WT0': WT0, 'WT1': WT1, 'YBASE': YBASE}
probcole = EM1D.EM1D(mesh1D, sigmaMap = Colemap, **options)
if probcole.ispaired:
    probcole.unpair()
if FDsurvey.ispaired:
    FDsurvey.unpair()
probcole.pair(FDsurvey)
probcole.chi = np.zeros(FDsurvey.nlay)
probcole.CondType = 'Complex'
HzCole = probcole.fields(m_1D)
Hzanal = EM1DAnal.Hzanal(sig_half, FDsurvey.frequency, FDsurvey.offset, 'secondary')

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8, 5))
ax.loglog(FDsurvey.frequency, abs(Hz.real), 'b')
ax.loglog(FDsurvey.frequency, abs(Hz.imag), 'r')
ax.loglog(FDsurvey.frequency, abs(HzCole.real), 'b*')
ax.loglog(FDsurvey.frequency, abs(HzCole.real), 'b--')
ax.loglog(FDsurvey.frequency, abs(HzCole.imag), 'r*')
legend(('Real $H_{z FHT}$', 'Real $H_{z anal}$', 'Imag $H_{z FHT}$', 'Imag $H_{z anal}$'))
ax.text(1e4, 1e-8, "$\\tau$ = 0.1", fontsize = 18)